In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import calendar
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from torch_geometric.loader import NeighborSampler as RawNeighborSampler
from torch_geometric.nn import SAGEConv

import numpy as np
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# output_path = "/run/media/yunchen/lacie"
#dataset_path = "./datasets/prewalk/with_amenity_filters"
dataset_path = "./datasets/prewalk/without_amenity_filters"
print(torch.cuda.is_available())

DECAY_ALPHA = -0.04
HIDDEN_LAYER = 16
NUM_LAYERS = 5
NEIGHBOUR_SIZE = [5,3,3,3,3]
DROP_OUT = 0.3
BATCH_SIZE = 256
SHUFFLE = True

True


In [5]:
street_nodes_df = pd.read_csv(f"{dataset_path}/akl_prewalked_nodes.csv")
street_edges_df = pd.read_csv(f"{dataset_path}/akl_street_edges.csv")

# 求倒数
# street_edges_df[['distance']] = street_edges_df[['distance']].rdiv(1)

# exponential weighted calculation
print(street_edges_df[['distance']])
print(street_edges_df[['distance']] * np.exp(DECAY_ALPHA * street_edges_df[['distance']] ))

street_edges_df[['distance']] = street_edges_df[['distance']] * np.exp(DECAY_ALPHA * street_edges_df[['distance']])
#street_edges_df.to_csv(f"./outputs/play_edge.csv")

source_street_index, targe_street_index, street_distance_weight = street_edges_df["source_street"], street_edges_df[
    "target_street"], street_edges_df["distance"]

street_edges_source_index_tensor = torch.tensor([source_street_index.values.tolist()])
street_edges_target_index_tensor = torch.tensor([targe_street_index.values.tolist()])
street_edges_index_tensor = torch.cat((street_edges_source_index_tensor, street_edges_target_index_tensor), 0)
street_edges_weight_tensor = torch.tensor(street_distance_weight.values.tolist())

#print(street_edges_index_tensor)
#print(street_edges_weight_tensor)

         distance
0          96.873
1          96.873
2         100.787
3         100.787
4         118.706
...           ...
1004713   310.398
1004714   328.777
1004715   328.777
1004716   493.528
1004717   493.528

[1004718 rows x 1 columns]
         distance
0        2.010696
1        2.010696
2        1.788772
3        1.788772
4        1.028817
...           ...
1004713  0.001258
1004714  0.000639
1004715  0.000639
1004716  0.000001
1004717  0.000001

[1004718 rows x 1 columns]


In [6]:
count_poi_df = street_nodes_df.copy()
#count_poi_df["poi_count"] = count_poi_df.apply(lambda row:row.amenity+row.restaurant+row.education+row.healthcare+row.shop+row.cloth,axis=1)
count_poi_df["poi_count"] = count_poi_df.apply(lambda row:row.amenity+row.restaurant+row.school+row.healthcare+row.shop+row.clothes,axis=1)
positive_nodes_with_poi_df_layer0 = count_poi_df[count_poi_df["poi_count"]>0]
negative_nodes_without_poi_df_layer0 = count_poi_df[count_poi_df["poi_count"]<=0]
positive_nodes_with_poi_df_layer1 = count_poi_df[count_poi_df["Layer_1_agg_poi_count"]>0]
negative_nodes_without_poi_df_layer1 = count_poi_df[count_poi_df["Layer_1_agg_poi_count"]<=0]
positive_nodes_with_poi_df_layer2 = count_poi_df[count_poi_df["Layer_2_agg_poi_count"]>0]
negative_nodes_without_poi_df_layer2 = count_poi_df[count_poi_df["Layer_2_agg_poi_count"]<=0]
positive_nodes_with_poi_df_layer3 = count_poi_df[count_poi_df["Layer_3_agg_poi_count"]>0]
negative_nodes_without_poi_df_layer3 = count_poi_df[count_poi_df["Layer_3_agg_poi_count"]<=0]
positive_nodes_with_poi_df_layer4 = count_poi_df[count_poi_df["Layer_4_agg_poi_count"]>0]
negative_nodes_without_poi_df_layer4 = count_poi_df[count_poi_df["Layer_4_agg_poi_count"]<=0]
positive_nodes_with_poi_df_layer5 = count_poi_df[count_poi_df["Layer_5_agg_poi_count"]>0]
negative_nodes_without_poi_df_layer5 = count_poi_df[count_poi_df["Layer_5_agg_poi_count"]<=0]
print(f"Layer 0 Positive POI {len(positive_nodes_with_poi_df_layer0)}  Negative POI {len(negative_nodes_without_poi_df_layer0)}")
print(f"Layer 1 Positive POI {len(positive_nodes_with_poi_df_layer1)}  Negative POI {len(negative_nodes_without_poi_df_layer1)}")
print(f"Layer 2 Positive POI {len(positive_nodes_with_poi_df_layer2)}  Negative POI {len(negative_nodes_without_poi_df_layer2)}")
print(f"Layer 3 Positive POI {len(positive_nodes_with_poi_df_layer3)}  Negative POI {len(negative_nodes_without_poi_df_layer3)}")
print(f"Layer 4 Positive POI {len(positive_nodes_with_poi_df_layer4)}  Negative POI {len(negative_nodes_without_poi_df_layer4)}")
print(f"Layer 5 Positive POI {len(positive_nodes_with_poi_df_layer5)}  Negative POI {len(negative_nodes_without_poi_df_layer5)}")

Layer 0 Positive POI 16148  Negative POI 442104
Layer 1 Positive POI 30711  Negative POI 427541
Layer 2 Positive POI 52050  Negative POI 406202
Layer 3 Positive POI 70259  Negative POI 387993
Layer 4 Positive POI 86345  Negative POI 371907
Layer 5 Positive POI 100598  Negative POI 357654


In [7]:
street_nodes_df_copy = street_nodes_df.copy()
# street_nodes_df_copy = street_nodes_df_copy[["Layer_5_agg_restaurant", "Layer_5_agg_amenity","Layer_5_agg_education","Layer_5_agg_healthcare","Layer_5_agg_shop","Layer_5_agg_cloth","Layer_5_agg_average_poi_distance"]]
#street_nodes_df_copy = street_nodes_df_copy[["restaurant","amenity","education","healthcare","shop","cloth","Average_POI_Distance","Layer_1_agg_restaurant", "Layer_1_agg_amenity","Layer_1_agg_education","Layer_1_agg_healthcare","Layer_1_agg_shop","Layer_1_agg_cloth","Layer_1_agg_average_poi_distance","Layer_2_agg_restaurant", "Layer_2_agg_amenity","Layer_2_agg_education","Layer_2_agg_healthcare","Layer_2_agg_shop","Layer_2_agg_cloth","Layer_2_agg_average_poi_distance","Layer_3_agg_restaurant", "Layer_3_agg_amenity","Layer_3_agg_education","Layer_3_agg_healthcare","Layer_3_agg_shop","Layer_3_agg_cloth","Layer_3_agg_average_poi_distance","Layer_4_agg_restaurant", "Layer_4_agg_amenity","Layer_4_agg_education","Layer_4_agg_healthcare","Layer_4_agg_shop","Layer_4_agg_cloth","Layer_4_agg_average_poi_distance","Layer_5_agg_restaurant", "Layer_5_agg_amenity","Layer_5_agg_education","Layer_5_agg_healthcare","Layer_5_agg_shop","Layer_5_agg_cloth","Layer_5_agg_average_poi_distance",]]

#street_nodes_df_copy = street_nodes_df_copy[["restaurant", "amenity","school","healthcare","shop","clothes"]]
#street_nodes_df_copy = street_nodes_df_copy[["Layer_5_agg_restaurant", "Layer_5_agg_amenity","Layer_5_agg_school","Layer_5_agg_healthcare","Layer_5_agg_shop","Layer_5_agg_clothes"]]#,"Layer_5_agg_average_poi_distance"]]
street_nodes_df_copy = street_nodes_df_copy[["restaurant","amenity","school","healthcare","shop","clothes","Average_POI_Distance","Layer_1_agg_restaurant", "Layer_1_agg_amenity","Layer_1_agg_school","Layer_1_agg_healthcare","Layer_1_agg_shop","Layer_1_agg_clothes","Layer_1_agg_average_poi_distance","Layer_2_agg_restaurant", "Layer_2_agg_amenity","Layer_2_agg_school","Layer_2_agg_healthcare","Layer_2_agg_shop","Layer_2_agg_clothes","Layer_2_agg_average_poi_distance","Layer_3_agg_restaurant", "Layer_3_agg_amenity","Layer_3_agg_school","Layer_3_agg_healthcare","Layer_3_agg_shop","Layer_3_agg_clothes","Layer_3_agg_average_poi_distance","Layer_4_agg_restaurant", "Layer_4_agg_amenity","Layer_4_agg_school","Layer_4_agg_healthcare","Layer_4_agg_shop","Layer_4_agg_clothes","Layer_4_agg_average_poi_distance","Layer_5_agg_restaurant", "Layer_5_agg_amenity","Layer_5_agg_school","Layer_5_agg_healthcare","Layer_5_agg_shop","Layer_5_agg_clothes","Layer_5_agg_average_poi_distance",]]
print(street_nodes_df_copy.columns)

scaler = MinMaxScaler()
street_nodes_df_transformed = scaler.fit_transform(street_nodes_df_copy)
street_nodes_features_tensor = torch.tensor(street_nodes_df_transformed)

number_of_nodes = len(street_nodes_features_tensor)
number_of_node_features = len(street_nodes_features_tensor[0])
print(street_nodes_features_tensor)
print(number_of_nodes)
print(number_of_node_features)

Index(['restaurant', 'amenity', 'school', 'healthcare', 'shop', 'clothes',
       'Average_POI_Distance', 'Layer_1_agg_restaurant', 'Layer_1_agg_amenity',
       'Layer_1_agg_school', 'Layer_1_agg_healthcare', 'Layer_1_agg_shop',
       'Layer_1_agg_clothes', 'Layer_1_agg_average_poi_distance',
       'Layer_2_agg_restaurant', 'Layer_2_agg_amenity', 'Layer_2_agg_school',
       'Layer_2_agg_healthcare', 'Layer_2_agg_shop', 'Layer_2_agg_clothes',
       'Layer_2_agg_average_poi_distance', 'Layer_3_agg_restaurant',
       'Layer_3_agg_amenity', 'Layer_3_agg_school', 'Layer_3_agg_healthcare',
       'Layer_3_agg_shop', 'Layer_3_agg_clothes',
       'Layer_3_agg_average_poi_distance', 'Layer_4_agg_restaurant',
       'Layer_4_agg_amenity', 'Layer_4_agg_school', 'Layer_4_agg_healthcare',
       'Layer_4_agg_shop', 'Layer_4_agg_clothes',
       'Layer_4_agg_average_poi_distance', 'Layer_5_agg_restaurant',
       'Layer_5_agg_amenity', 'Layer_5_agg_school', 'Layer_5_agg_healthcare',
       'L

In [8]:
# street_nodes_df = street_nodes_df[street_nodes_df.columns[4:]]
#
# street_nodes_df_copy = street_nodes_df.copy()
# print(street_nodes_df_copy.head)
# # street_nodes_df_copy.drop(["street_length", "Average_POI_Distance","x","y"], axis=1, inplace=True)
# street_nodes_df_copy.drop(["street_length","Average_POI_Distance"], axis=1, inplace=True)
# print(street_nodes_df_copy.columns)
#
# street_nodes_features_tensor = torch.tensor(street_nodes_df_copy.values.tolist())
#
# number_of_nodes = len(street_nodes_features_tensor)
# number_of_node_features = len(street_nodes_features_tensor[0])
# print(street_nodes_features_tensor)
# print(number_of_nodes)
# print(number_of_node_features)
#
# street_nodes_df_copy_2 = street_nodes_df.copy()
# # street_nodes_df_copy_2["poi_count"] = street_nodes_df_copy_2.apply(lambda row:row.amenity+row.restaurant+row.school+row.shop+row.healthcare+row.clothes,axis=1)
# street_nodes_df_copy_2["poi_count"] = street_nodes_df_copy_2.apply(lambda row:row.amenity+row.restaurant+row.education+row.healthcare+row.shop+row.cloth,axis=1)
# positive_nodes_with_poi_df_layer = street_nodes_df_copy_2[street_nodes_df_copy_2["poi_count"]>0]
# negative_nodes_without_poi_df_layer = street_nodes_df_copy_2[street_nodes_df_copy_2["poi_count"]<=0]
# positive_nodes_index = torch.tensor(positive_nodes_with_poi_df.index)


In [9]:
# def custom_pos_sampling_with_POI(
#         edge_weight: Tensor,
#         batch: Tensor,
# ):
#     pos_node_seq = []
#     neg_node_seq = []
#     for start_node_id in batch:
#         current_node_seq = [start_node_id.item()]
#         current_node_id = current_node_seq[-1]
#
#         neighbours_edge_index = (street_edges_index_tensor == current_node_id).nonzero(as_tuple=True)[1]
#         len_neighbours_edge_index = len(neighbours_edge_index)
#         neighbour_id_list = []
#         for i in neighbours_edge_index:
#             neighbour_id_list.append(select_neighbour_node_id_from_edge(i,current_node_id))
#
#         normalized_neighbour_poi_weights = calculate_normalized_poi_weights(neighbour_id_list)
#
#         global_node_without_poi = negative_nodes_without_poi_df.sample(replace=True).index.values[0]
#         global_node_with_poi = positive_nodes_with_poi_df.sample(n=1,replace=True).index.values[0]
#         # all neighbours don't have POI
#         if np.isnan(normalized_neighbour_poi_weights).all():
#             # Also no neighbour
#             if len_neighbours_edge_index == 0:
#                 pos_node_seq.append([current_node_id,global_node_without_poi])
#                 neg_node_seq.append(global_node_with_poi)
#                 # neg_node_seq.append(current_node_id)
#                 # pos_node_seq.append([global_node_with_poi,global_node_with_poi])
#                 continue
#             # No neighbour has POIs but do have neighbours
#             else:
#                 # Pick a neighbour randomly using distance distribution to give future poi opportunity
#                 neighbour_distance_weights = torch.index_select(edge_weight, 0, neighbours_edge_index).numpy()
#                 norm_neighbour_distance_weights = calculate_normalized_distance_weights(neighbour_distance_weights)
#                 neighbour_weights_index = np.random.choice(len(norm_neighbour_distance_weights),p=norm_neighbour_distance_weights)
#
#                 pos_node_seq.append([current_node_id,global_node_without_poi])
#                 neg_node_seq.append(global_node_with_poi)
#                 continue
#         # normal case pick with poi distribution
#         else:
#             neighbour_weights_index = np.random.choice(len(normalized_neighbour_poi_weights),
#                                                        p=normalized_neighbour_poi_weights)
#
#         next_edge_index = neighbours_edge_index[neighbour_weights_index]
#         next_node_id = select_neighbour_node_id_from_edge(next_edge_index,current_node_id)
#
#         pos_node_seq.append([current_node_id,next_node_id])
#         neg_node_seq.append(global_node_without_poi)
#
#     return torch.from_numpy(np.asarray(pos_node_seq, dtype=np.int32))[:, 1], torch.from_numpy(
#         np.asarray(neg_node_seq, dtype=np.int32))
#
# def calculate_normalized_distance_weights(neighbour_distance_weights):
#     neighbour_distance_weights_sum = sum(neighbour_distance_weights)
#     reverted_norm_neighbour_distance_weights = [1-(i / neighbour_distance_weights_sum) for i in neighbour_distance_weights]
#
#     reverted_norm_neighbour_distance_weights_sum = sum(reverted_norm_neighbour_distance_weights)
#     norm_neighbour_distance_weights = [(i / reverted_norm_neighbour_distance_weights_sum) for i in reverted_norm_neighbour_distance_weights]
#     return norm_neighbour_distance_weights
#
# def calculate_normalized_poi_weights(neighbour_id_list):
#     neighbour_id_weights = []
#     for neighbour_id in neighbour_id_list:
#         poi_weight = 0
#         neighbour_features = torch.index_select(street_nodes_features_tensor, 0,
#                                                 torch.tensor(int(neighbour_id), dtype=torch.int32))
#         poi_weight += torch.sum(neighbour_features)
#         neighbour_id_weights.append(poi_weight)
#
#     neighbour_poi_weights = np.array(neighbour_id_weights)
#     neighbour_poi_weights_sum=sum(neighbour_poi_weights)
#     normalized_neighbour_poi_weights = [i / neighbour_poi_weights_sum for i in neighbour_poi_weights]
#     return normalized_neighbour_poi_weights
#
# def select_neighbour_node_id_from_edge(next_edge_index,current_node_id):
#     next_edge_df = street_edges_df.iloc[[next_edge_index]]
#     next_edge = next_edge_df.values[0]
#     if next_edge[0] != current_node_id:
#         neighbour_node_id = next_edge[0]
#     else:
#         neighbour_node_id = next_edge[1]
#
#     return neighbour_node_id


In [10]:
def custom_sampling_with_Prewalk(
        edge_weight: Tensor,
        batch: Tensor,
):
    pos_node_seq = []
    neg_node_seq = []
    for start_node_id in batch:
        current_node_seq = [start_node_id.item()]
        current_node_id = current_node_seq[-1]

        neighbours_edge_index = (street_edges_index_tensor == current_node_id).nonzero(as_tuple=True)[1]
        neighbour_id_list = []
        for i in neighbours_edge_index:
            neighbour_id_list.append(select_neighbour_node_id_from_edge(i,current_node_id))

        normalized_neighbour_poi_weights = calculate_normalized_poi_weights(neighbour_id_list)

        global_node_without_poi_layer5 = negative_nodes_without_poi_df_layer5.sample(replace=True).index.values[0]
        global_node_with_poi_layer5 = positive_nodes_with_poi_df_layer5.sample(replace=True).index.values[0]
        # all neighbours don't have POI
        if np.isnan(normalized_neighbour_poi_weights).all():
            pos_node_seq.append([current_node_id,global_node_without_poi_layer5])
            neg_node_seq.append(global_node_with_poi_layer5)
            continue
        # normal case pick with poi distribution
        else:
            neighbour_weights_index = np.random.choice(len(normalized_neighbour_poi_weights),
                                                       p=normalized_neighbour_poi_weights)

        next_edge_index = neighbours_edge_index[neighbour_weights_index]
        next_node_id = select_neighbour_node_id_from_edge(next_edge_index,current_node_id)

        pos_node_seq.append([current_node_id,next_node_id])
        neg_node_seq.append(global_node_without_poi_layer5)

    return torch.from_numpy(np.asarray(pos_node_seq, dtype=np.int32))[:, 1], torch.from_numpy(
        np.asarray(neg_node_seq, dtype=np.int32))

def calculate_normalized_poi_weights(neighbour_id_list):
    neighbour_id_weights = []
    for neighbour_id in neighbour_id_list:
        poi_weight = 0
        neighbour_features = torch.index_select(street_nodes_features_tensor, 0,
                                                torch.tensor(int(neighbour_id), dtype=torch.int32))
        neighbour_features = neighbour_features[0][len(neighbour_features)-6:]

        poi_weight += torch.sum(neighbour_features)
        neighbour_id_weights.append(poi_weight)

    neighbour_poi_weights = np.array(neighbour_id_weights)
    neighbour_poi_weights_sum=sum(neighbour_poi_weights)
    normalized_neighbour_poi_weights = [i / neighbour_poi_weights_sum for i in neighbour_poi_weights]
    return normalized_neighbour_poi_weights

def select_neighbour_node_id_from_edge(next_edge_index,current_node_id):
    next_edge_df = street_edges_df.iloc[[next_edge_index]]
    next_edge = next_edge_df.values[0]
    if next_edge[0] != current_node_id:
        neighbour_node_id = next_edge[0]
    else:
        neighbour_node_id = next_edge[1]

    return neighbour_node_id


In [11]:
"""
RawNeighborSampler This module iteratively samples neighbors (at each layer) and constructs bipartite graphs that simulate the actual computation flow of GNNs.

format-selected)
NeighborSampler holds the current :obj:batch_size, the IDs :obj:n_id of all nodes involved in the computation, and a list of bipartite graph objects via the tuple :obj:(edge_index, e_id, size), where :obj:edge_index represents the bipartite edges between source and target nodes, :obj:e_id denotes the IDs of original edges in the full graph, and :obj:size holds the shape of the bipartite graph.

The actual computation graphs are then returned in reverse-mode, meaning that we pass messages from a larger set of nodes to a smaller one, until we reach the nodes for which we originally wanted to compute embeddings.
https://www.arangodb.com/2021/08/a-comprehensive-case-study-of-graphsage-using-pytorchgeometric/
https://towardsdatascience.com/pytorch-geometric-graph-embedding-da71d614c3a
https://gist.github.com/anuradhawick/904e7f2d2101f4b76516d04046007426
https://zhuanlan.zhihu.com/p/387262710
"""

class NeighborSampler(RawNeighborSampler):
    def sample(self, batch):
        batch = torch.tensor(batch)

        pos_batch, neg_batch = custom_sampling_with_Prewalk(street_edges_weight_tensor, batch)
        batch = torch.cat([batch, pos_batch, neg_batch], dim=0)
        return super(NeighborSampler,self).sample(batch)

train_loader = NeighborSampler(street_edges_index_tensor, sizes=NEIGHBOUR_SIZE, batch_size=BATCH_SIZE, num_nodes=number_of_nodes,shuffle=SHUFFLE)

In [12]:
from typing import List, Optional, Tuple, Union

import torch.nn.functional as F
from torch import Tensor
from torch.nn import LSTM

from torch_geometric.nn.aggr import Aggregation, MultiAggregation
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor,OptTensor, Size


class CustomSAGEConv(MessagePassing):
    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        aggr: Optional[Union[str, List[str], Aggregation]] = "mean",
        normalize: bool = False,
        root_weight: bool = True,
        project: bool = False,
        bias: bool = True,
        **kwargs,
    ):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.root_weight = root_weight
        self.project = project

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        if aggr == 'lstm':
            kwargs.setdefault('aggr_kwargs', {})
            kwargs['aggr_kwargs'].setdefault('in_channels', in_channels[0])
            kwargs['aggr_kwargs'].setdefault('out_channels', in_channels[0])

        super().__init__(aggr, **kwargs)

        if self.project:
            self.lin = Linear(in_channels[0], in_channels[0], bias=True)

        if self.aggr is None:
            self.fuse = False  # No "fused" message_and_aggregate.
            self.lstm = LSTM(in_channels[0], in_channels[0], batch_first=True)

        if isinstance(self.aggr_module, MultiAggregation):
            aggr_out_channels = self.aggr_module.get_out_channels(
                in_channels[0])
        else:
            aggr_out_channels = in_channels[0]

        self.lin_l = Linear(aggr_out_channels, out_channels, bias=bias)
        if self.root_weight:
            self.lin_r = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        if self.project:
            self.lin.reset_parameters()
        self.aggr_module.reset_parameters()
        self.lin_l.reset_parameters()
        if self.root_weight:
            self.lin_r.reset_parameters()

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                size: Size = None,edge_weight: OptTensor=None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        if self.project and hasattr(self, 'lin'):
            x = (self.lin(x[0]).relu(), x[1])

        # propagate_type: (x: OptPairTensor)
        edge_weight: OptTensor = edge_weight
        out = self.propagate(edge_index, x=x, size=size,edge_weight=edge_weight)
        out = self.lin_l(out)

        x_r = x[1]
        if self.root_weight and x_r is not None:
            out = out + self.lin_r(x_r)

        if self.normalize:
            out = F.normalize(out, p=2., dim=-1)

        return out

    def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
        #print(f"In Message Edge Weight {edge_weight}")
        #print(f"In Message Edge Weight view  {edge_weight.view(-1, 1)}")
        #print(f"In Message x_j  {x_j}")
        #print(f"In Message {edge_weight.view(-1, 1) * x_j}")
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, aggr={self.aggr})')

In [13]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(CustomSAGEConv(in_channels, hidden_channels))

    def forward(self, x, adjs):
        for i, (edge_index, edge_id, size) in enumerate(adjs):
            copy_edge_weight = street_edges_weight_tensor.clone()
            nn_edge_weight =copy_edge_weight[edge_id].to(device)
            x_target = x[:size[1]]  # Target nodes are always placed first.

            # if i > DISTANCE_PENALTY_LAYER:
            x = self.convs[i]((x, x_target), edge_index, edge_weight = nn_edge_weight)
            # else:
            #     x = self.convs[i]((x, x_target), edge_index)

            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=DROP_OUT, training=self.training)
        return x

    def full_forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=DROP_OUT, training=self.training)
        return x


model = SAGE(number_of_node_features, hidden_channels=HIDDEN_LAYER, num_layers=NUM_LAYERS)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

edge_weight_tensor = street_edges_weight_tensor.clone()
x, edge_index = street_nodes_features_tensor.to(device), street_edges_index_tensor.to(device)

In [14]:
def regression_train(embedding_df):
    AKL_df = pd.read_csv(f"{dataset_path}/property_data_with_street.csv", encoding='latin1')
    AKL_df = AKL_df.drop(['Unnamed: 0'], axis=1)

    akl_embedding_np = embedding_df.numpy()  #convert to Numpy array
    akl_embedding_df = pd.DataFrame(akl_embedding_np)  #convert to a dataframe
    embedding_size = akl_embedding_df.shape[1]
    akl_embedding_df.columns = ['street_embedding_' + str(i) for i in range(embedding_size)]

    akl_street_nodes_df = pd.read_csv(f"{dataset_path}/akl_prewalked_nodes.csv")
    akl_street_nodes_df = akl_street_nodes_df.rename(columns={"source": "street_sources", "target": "street_targets"})
    AKL_df = find_embedding_for_property(AKL_df, akl_street_nodes_df, akl_embedding_df)
    property_columns = ['CL_Suburb', 'CL_Sale_Tenure', 'CL_Sale_Date', 'CL_Land_Valuation_Capital_Value',
                        'CL_Building_Floor_Area', 'CL_Building_Site_Cover',
                        'CL_Land_Area', 'CL_Bldg_Const', 'CL_Bldg_Cond', 'CL_Roof_Const', 'CL_Roof_Cond',
                        'CL_Category', 'CL_LUD_Age', 'CL_LUD_Land_Use_Description',
                        'CL_MAS_No_Main_Roof_Garages', 'CL_Bedrooms', 'CL_Bathrooms'] + ['street_embedding_' + str(i)
                                                                                         for i in range(embedding_size)]
    X_columns = AKL_df[property_columns].values
    #print(X_columns)
    Y_column = AKL_df['Log_Sale_Price_Net'].values

    X_train, X_test, Y_train, Y_test = train_test_split(X_columns, Y_column, test_size=0.2, random_state=1,
                                                        shuffle=True)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1, shuffle=True)
    hedonic_regression = LinearRegression()
    hedonic_regression.fit(X_train, Y_train)

    hedonic_regression_validation_result = hedonic_regression.predict(X_val)

    validation_RMSE = round(mean_squared_error(Y_val, hedonic_regression_validation_result), 4)
    validation_R2 = round(r2_score(Y_val, hedonic_regression_validation_result), 4)
    return validation_RMSE, validation_R2


def find_embedding_for_property(property_df, street_df, emb_df):
    street_with_embedding = street_df.merge(emb_df, left_index=True, right_index=True)
    output_df = property_df.merge(street_with_embedding, on=["street_sources", "street_targets"])
    return output_df


In [15]:
def train():
    model.train()

    total_loss = 0
    # i=0
    for batch_size, n_id, adjs in train_loader:
        # i+=1
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()
        out = model(x[n_id].float().to(device), adjs)
        out, pos_out, neg_out = out.split(out.size(0) // 3, dim=0)

        pos_loss = F.logsigmoid((out * pos_out).sum(-1)).mean()
        neg_loss = F.logsigmoid(-(out * neg_out).sum(-1)).mean()
        loss = -pos_loss - neg_loss

        loss.backward()
        optimizer.step()

        total_loss += float(loss) * out.size(0)
        # print(i)
    return total_loss / number_of_nodes


@torch.no_grad()
def get_model_embedding():
    model.eval()
    embedding = model.full_forward(x.float().to(device), edge_index.to(device)).cpu()
    return embedding


historical_rmse, historical_r2, historical_loss = np.inf, np.inf, np.inf
for epoch in range(1, 80):
    loss = train()
    model.eval()

    with torch.no_grad():
        temp_embedding = model.full_forward(x.float().to(device), edge_index.to(device)).cpu()
        regression_rmse, regression_r2 = regression_train(temp_embedding)

        if historical_rmse == np.inf:
            historical_rmse = regression_rmse

        if historical_r2 == np.inf:
            historical_r2 = regression_r2

        if historical_loss == np.inf:
            historical_loss = loss

        if loss > historical_loss:
            if regression_rmse > historical_rmse:
                if epoch < 10:
                    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Current RMSE is {regression_rmse}, Current R2 is {regression_r2} ')
                    continue
                print(
                    f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Stopped on RMSE! Current RMSE is {regression_rmse}, previous RMSE is {historical_rmse} ')
                break

            if regression_r2 < historical_r2:
                 if epoch < 10:
                    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Current RMSE is {regression_rmse}, Current R2 is {regression_r2} ')
                    continue
                 print(
                     f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Stopped on r2! Current R2 is {regression_r2}, previous R2 is {historical_r2} ')
                 break
        else:
            historical_loss = loss
            historical_rmse = regression_rmse
            historical_r2 = regression_r2
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Current RMSE is {regression_rmse}, Current R2 is {regression_r2} ')
        output_embedding = temp_embedding

#output_embedding = get_model_embedding()

Epoch: 001, Loss: 2.6568,Current RMSE is 0.0606, Current R2 is 0.7943 
Epoch: 002, Loss: 1.1019,Current RMSE is 0.0605, Current R2 is 0.7946 
Epoch: 003, Loss: 1.0570,Current RMSE is 0.0607, Current R2 is 0.7941 


KeyboardInterrupt: 

In [ ]:
print(output_embedding)

In [ ]:
output_np = output_embedding.numpy()  #convert to Numpy array
output_df = pd.DataFrame(output_np)  #convert to a dataframe
current_GMT = time.gmtime()
ts = calendar.timegm(current_GMT)
output_df.to_csv(f"./outputs/embeddings/prewalk/without_amenity_filters/edge_weight/akl_embedding_{ts}.csv", index=False)  #save to file
print(f"akl_embedding_{ts}.csv")